In [1]:
from Readfile import Read
from astropy.table import QTable
import numpy as np
import astropy.units as u

In [11]:
def ComponentMass(filename, particletype):
#Inputs:
    #filename = name of file
    #particletype = kind of particle i.e. Halo type (1), Disk type (2), Bulge type (3)
#Returns:
    #return total mass (rounded to 3 decimal places) of any desired galaxy component in units of 10^12 Msun 
    #i.e. total mass of disk stars, halo stars, etc.)

    #read in file and extract needed data from it
    time, total, data = Read(filename)
    
    #create index to store data only for wanted particle type
    index = np.where(data['type'] == particletype)
    
    #create array to store mass values for wanted particle type
    mass = data['m'][index]
    
    #in table it is already in units of 10^10 Msun so divide by 10^2 to get units of 10^12 Msun
    Msun12units = (10**-2) * u.Msun 
    
    #sum up all mass values in array to get total mass of the component
    totalmass = np.sum(mass)
    
    #round total mass to 3 decimal places
    roundedmass = np.around(totalmass, 3) * Msun12units
    
    #return total component mass rounded to 3 decimal places returned without units 
    return roundedmass

In [12]:
#testing ComponentMass function by calculating disk mass of MW 
MWtype2mass = ComponentMass("/Users/Maddog/Astr400b/MW_000.txt", 2.0)
print(MWtype2mass)

0.075 solMass


In [8]:
#create table to store results of Galaxy Mass in
#columns should be: Galaxy Name, Halo Mass (units), Disk Mass (units), Bulge Mass (units), Total Mass (units), fbar
#rows are for each galaxy
#mass is in units of 10^12 Msun

#establish array for different particle types 1.0 = halo, 2.0 = disk, 3.0 = bulge
particletypes = [1.0, 2.0, 3.0]

#loop through particle types to compute component mass for MW, M31, and M33
#I realize this seems to be a bit more work than just typing out the number of the particle type by why not be fancy?
for particle in particletypes:
    if particle == 1.0:
        MWhalomass = ComponentMass("/Users/Maddog/Astr400b/MW_000.txt", particle)   #Halo component of MW
        M31halomass = ComponentMass("/Users/Maddog/Astr400b/M31_000.txt", particle)  #Halo component of M31
        M33halomass = ComponentMass("/Users/Maddog/Astr400b/M33_000.txt", particle)  #Halo component of M33
    if particle == 2.0:
        MWdiskmass = ComponentMass("/Users/Maddog/Astr400b/MW_000.txt", particle)  #Disk component of MW
        M31diskmass = ComponentMass("/Users/Maddog/Astr400b/M31_000.txt", particle)  #Disk component of M31
        M33diskmass = ComponentMass("/Users/Maddog/Astr400b/M33_000.txt", particle)  #Disk component of M33
    if particle == 3.0:
        MWbulgemass = ComponentMass("/Users/Maddog/Astr400b/MW_000.txt", particle)  #Bulge component of MW
        M31bulgemass = ComponentMass("/Users/Maddog/Astr400b/M31_000.txt", particle)  #Bulge component of M31
        M33bulgemass = 0.0   #M33 has no bulge

#compute total mass of each galaxy by adding all mass components together
totalMWmass = MWhalomass + MWdiskmass + MWbulgemass   #total mass of Milky Way
totalM31mass = MWhalomass + M31diskmass + M31bulgemass   #total mass of M31
totalM33mass = M33halomass + M33diskmass + M33bulgemass   #total mass of M33

#compute total mass components of LG
LGhalomass = MWhalomass + MWhalomass + M33halomass   #sum of halo components of local group
LGdiskmass = MWdiskmass + M31diskmass + M33diskmass  #sum of disk components of local group
LGbulgemass = MWbulgemass + M31bulgemass + M33bulgemass  #sum of bulge components of local group

#compute total mass of local group by adding together totalmasses of each galaxy calculated above
totalLGmass = totalMWmass + totalM31mass + totalM33mass

#compute baryon fraction for each galaxy where fbar = total stellar mass / total mass
MWfbar = (MWdiskmass + MWbulgemass)/totalMWmass   #baryon fraction of Milky Way
M31fbar = (M31diskmass + M31bulgemass)/totalM31mass   #baryon fraction of M31
M33fbar = (M33diskmass + M33bulgemass)/totalM33mass   #baryon fraction of M33
LGfbar = (MWdiskmass + MWbulgemass + M31diskmass + M31bulgemass + M33diskmass + M33bulgemass)/totalLGmass  #baryon fraction of Local Group

#set units of mass
Msununits = u.M_sun

#create arrays to store wanted quantities in columns of table
galaxynames = ["Milky Way", "M31", "M33", "Local Group"]    #array to store galaxy names
halomasses = [MWhalomass, M31halomass, M33halomass, LGhalomass]*Msununits    #array to store halo mass components of galaxies
diskmasses = [MWdiskmass, M31diskmass, M33diskmass, LGdiskmass]*Msununits    #array to store disk mass components of galaxies
bulgemasses = [MWbulgemass, M31bulgemass, M33bulgemass, LGbulgemass]*Msununits   #array to store bulge mass components of galaxies
totalmasses = [totalMWmass, totalM31mass, totalM33mass, totalLGmass]*Msununits   #array to store total masses of each galaxy
fbar = [MWfbar, M31fbar, M33fbar, LGfbar]   #array to store fbar ratios for each galaxy as well as LG

#make table to store all values in
t = QTable([galaxynames, halomasses, diskmasses, bulgemasses, totalmasses, fbar], names=('Galaxy Name', 'Halo Mass (10^12 Msun)', 'Disk Mass (10^12 Msun)', 'Bulge Mass (10^12 Msun)', "Total (10^12 Msun)", "fbar"), meta={'name': 'Local Group Mass Table'})

In [9]:
t

Galaxy Name,Halo Mass (10^12 Msun),Disk Mass (10^12 Msun),Bulge Mass (10^12 Msun),Total (10^12 Msun),fbar
,solMass,solMass,solMass,solMass,
str11,float64,float64,float64,float64,float64
Milky Way,1.97492,0.075,0.01,2.05992,0.04126373839760767
M31,1.92088,0.12,0.01905,2.11397,0.06577671395525954
M33,0.18661000000000003,0.009300000000000001,0.0,0.19591000000000003,0.04747077739778469
Local Group,4.13645,0.2043,0.02905,4.3698,0.05340061330037989


In [10]:
#Although this table is easy to make, there's no easy way to save it as a pdf.  
#SO I've decided to make the table in LateX instead as well as answer the questions so that I may save it as a pdf
#print text to be added to table
for i in range(len(galaxynames)):
    print(galaxynames[i], np.around(halomasses[i],3), np.around(diskmasses[i],3), np.around(bulgemasses[i],3), np.around(totalmasses[i],3), np.around(fbar[i],3))

Milky Way 1.975 solMass 0.075 solMass 0.01 solMass 2.06 solMass 0.041
M31 1.921 solMass 0.12 solMass 0.019 solMass 2.114 solMass 0.066
M33 0.187 solMass 0.009 solMass 0.0 solMass 0.196 solMass 0.047
Local Group 4.136 solMass 0.204 solMass 0.029 solMass 4.37 solMass 0.053
